# Поиск наиболее близкого имени участника

In [14]:
# conda install -c conda-forge python-levenshtein
# или pip install python-levenshtein

import Levenshtein

In [67]:
import pandas as pd

In [141]:
# что было написано в заявке

claim_list = [
    'Шехавцова Анна',
    'Гречихина Наталья',
    'Козлова Алена',
    'Груздева Алина',
    'Кущенко Анна',
    'Чистякова Анастасия'
]

In [142]:
# результат расшифровки речи диктора

speech_recognition = [
    'кучменко она',
    'кущенко оксана',
    'груздь алина',
    'рычихина наталья',
    'шиховцева на',
    'чистова анастасия'
]

In [157]:
for recognized_name in speech_recognition:
    distances = {real_name: Levenshtein.distance(recognized_name, real_name)
                     for real_name in claim_list}
    #print(distances)
    
    break

In [144]:
distances

{'Шехавцова Анна': 11,
 'Гречихина Наталья': 14,
 'Козлова Алена': 11,
 'Груздева Алина': 10,
 'Кущенко Анна': 5,
 'Чистякова Анастасия': 15}

In [148]:
distances_df = pd.DataFrame(distances.items())
distances_df.columns = ['real_name', 'dista']

In [63]:
list_ = []
for recognized_name in speech_recognition:
    for real_name in claim_list:
        list_ += [[recognized_name, real_name, Levenshtein.distance(recognized_name, real_name)]]

In [146]:
df = pd.DataFrame(list_)
df.columns = ['recognized_name', 'real_name', 'dista']

In [182]:
merg = pd.merge(left=df, right=distances_df, how='inner', on = ['real_name', 'dista'])

In [150]:
merg

,recognized_name,real_name,dista
0,кучменко она,Шехавцова Анна,11
1,кущенко оксана,Шехавцова Анна,11
2,груздь алина,Шехавцова Анна,11
3,кучменко она,Гречихина Наталья,14
4,груздь алина,Гречихина Наталья,14
5,шиховцева на,Гречихина Наталья,14
6,кучменко она,Козлова Алена,11
7,кущенко оксана,Козлова Алена,11
8,кучменко она,Груздева Алина,10
9,кучменко она,Кущенко Анна,5


In [154]:
distances_df[distances_df.real_name=='Шехавцова Анна']

,real_name,dista
0,Шехавцова Анна,11


In [153]:
df[df.real_name=='Шехавцова Анна']

,recognized_name,real_name,dista
0,кучменко она,Шехавцова Анна,11
6,кущенко оксана,Шехавцова Анна,11
12,груздь алина,Шехавцова Анна,11
18,рычихина наталья,Шехавцова Анна,14
24,шиховцева на,Шехавцова Анна,6
30,чистова анастасия,Шехавцова Анна,13


In [155]:
merg[merg.real_name=='Шехавцова Анна']

,recognized_name,real_name,dista
0,кучменко она,Шехавцова Анна,11
1,кущенко оксана,Шехавцова Анна,11
2,груздь алина,Шехавцова Анна,11


In [158]:
#Мне не нравится что в рекомендациях = 11, при этом должно быть 6 + на Шехавцова Анна срзу видно что чтото не так

In [180]:
#Переделаем базовый справочник сравнения через группировку кросса по мин значению дистанции
distances_df2 = df.groupby(['recognized_name'])['dista'].min()
distances_df2.columns = ['recognized_name', 'dista']
distances_df2

recognized_name
груздь алина         5
кучменко она         5
кущенко оксана       5
рычихина наталья     3
чистова анастасия    4
шиховцева на         6
Name: dista, dtype: int64

In [179]:
#Джоиним по рекомендованному имени и дистанцйии
merg2 = pd.merge(left=df, right=distances_df2, how='inner', on = ['recognized_name', 'dista'])
merg2

,recognized_name,real_name,dista
0,кучменко она,Кущенко Анна,5
1,кущенко оксана,Кущенко Анна,5
2,груздь алина,Груздева Алина,5
3,рычихина наталья,Гречихина Наталья,3
4,шиховцева на,Шехавцова Анна,6
5,чистова анастасия,Чистякова Анастасия,4


In [181]:
#Вот теперь красота - на против каждого записанного имено идет реальное имя и они реально похожи и правдивы